In [1]:
import numpy as np
import tensorflow as tf
from pinn import get_network
from pinn.utils import pi_named, atomic_dress, connect_dist_grad
from pinn.models.base import export_model, get_train_op, MetricsCollector
import os, warnings
import tensorflow as tf
from glob import glob
from ase.collections import g2
from pinn.io import load_qm9, sparse_batch
from pinn import get_model, get_calc
from pinn.models.potential import make_metrics
from pinn.optimizers import get
from tensorflow import keras

In [2]:
filelist = glob('/Users/miguelnavaharris/Project/QM9/*.xyz')
dataset = lambda: load_qm9(filelist, splits={'train':8, 'test':2}) #lambda is being used here to assign execution of load_qm9 to the variable "dataset"
train = dataset()['train'].shuffle(1000).apply(sparse_batch(256))
test = dataset()['test'].apply(sparse_batch(256))

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-04-25 17:47:33.276899: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-25 17:47:33.277242: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
params = {'optimizer': {'class_name': 'Adam', 'config': {'learning_rate': {'class_name': 'ExponentialDecay', 'config': {'initial_learning_rate': 0.0003, 'decay_steps': 10000, 'decay_rate': 0.994}}, 'clipnorm': 0.01}}, 'model_dir': '/Users/miguelnavaharris/Project/miguelmodels/isthisstillsaved', 'network': {'name': 'PiNet', 'params': {'depth': 4, 'rc': 4.0, 'atom_types': [1, 6, 7, 8, 9]}}, 'model': {'name': 'potential_model', 'params': {'learning_rate': 0.001}}}

In [4]:
default_params = {
    ### Scaling and units # The loss function will be MSE((pred - label) * scale)
    # For vector/tensor predictions
    # the error will be per-component instead of per-atom
    # e_unit is the unit of energy to report w.r.t the input labels
    # no f_unit yet, f_unit is just e_unit/input coordinate unit
    # e.g. if one have input in Hartree, scales it by 100 for training
    #      and output eV when report error
    #      then e_scale should be 100, and e_unit = hartree2evp
    'e_dress': {},  # element-specific energy dress
    'e_scale': 1.0,  # energy scale for prediction
    'e_unit': 1.0,  # output unit of energy during prediction
    # Loss function options
    ## Energy
    'max_energy': False,        # if set to float, omit energies larger than it
    'use_e_per_atom': False,    # use e_per_atom to calculate e_loss
    'log_e_per_atom': False,    # log e_per_atom and its distribution
                                # ^- this is forcely done if use_e_per_atom
    'use_e_weight': False,      # scales the loss according to e_weight
    ## Force
    'use_force': False,         # include force in loss function
    'max_force_comp': False,    # if set to float, omit forces components larger than it
    'no_force_comp': False,     # if set to int, use as maximum number of force components for a update
    'use_f_weight': False,      # scales the loss according to f_weights
    ## Stress
    'use_stress': False,        # include stress in Loss function
    ## L2
    'use_l2': False,
    ## Loss function multipliers
    'e_loss_multiplier': 1.0,
    'f_loss_multiplier': 1.0,
    's_loss_multiplier': 1.0,
    'l2_loss_multiplier': 1.0,
    'separate_errors': False,   # workaround at this point
}

In [5]:
network = get_network(params['network'])
print(network)

In [6]:
model_params = default_params.copy()
model_params.update(params['model']['params'])

In [7]:
for batch in train:
    batch = network.preprocess(batch)
    connect_dist_grad(batch)
for batch in test:
    batch = network.preprocess(batch)
    connect_dist_grad(batch)


2022-04-25 17:47:38.029436: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
def get_compiled_model():
    model = network
    model.compile(
        optimizer=get(params['optimizer']),
        loss=keras.losses.mse,
        metrics=[keras.metrics.RootMeanSquaredError],
    )
    return model

In [9]:
model = get_compiled_model()
model.fit(train, epochs=2,  validation_data=test)

TypeError: 'property' object is not iterable